## Commissioner-Level RTT Data Loader (Incomplete & DTA)

This script processes RTT (Referral to Treatment) data for NHS commissioners (National, Region, and ICB levels) and loads it into the `rtt_data` table in PostgreSQL.

### Functionality:
- **User Input**: The script accepts a file path, year, month, and pathway type (`'Incomplete'` or `'DTA'`).
- **Sheet Handling**: Automatically iterates through predefined sheets depending on the selected pathway type, ignoring irrelevant ones.
- **Cleaning**:
  - Skips national rows from Region/ICB sheets to prevent duplication.
  - Parses and converts percentage strings into float values.
- **Transformation**: Data is melted into long format and enriched with metadata (year, month, pathway type, geo level).
- **Atomic Insert**: Ensures all sheets are processed and merged before a single DB write is attempted — enabling all-or-nothing integrity.

This script ensures a consistent and clean ingestion pipeline for all commissioner-level RTT performance data, ready for querying and visualization.


In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError

In [ ]:
# === USER INPUT ===
FILE_PATH = '../data/commissioner/Incomplete_apr2023_march_2024/Incomplete-Commissioner-12-Mar24.xlsx'
YEAR = 2024
MONTH = 3
PATHWAY_TYPE = 'DTA'  # 'Incomplete', 'DTA'

# === METRIC LOOKUP TABLE ===
METRICS_LOOKUP = {
    'Incomplete': [
        'Total number of incomplete pathways',
        '% within 18 weeks',
        'Average (median) waiting time (in weeks)',
        '92nd percentile waiting time (in weeks)',
        'Total 52 plus weeks'
    ],
    'DTA': [
        'Total number of incomplete pathways with a decision to admit for treatment',
        '% of incomplete pathways with a decision to admit for treatment'
    ]
}

# === DB CONNECTION ===
engine = create_engine("postgresql://postgres:<password>@localhost:5432/nhs_dashboard")

# === SHEET CONFIG ===
SHEETS = [
    ('National', 'National', 'NAT', 'NHS ENGLAND'),
    ('National with DTA', 'National', 'NAT', 'NHS ENGLAND'),
    ('Region', 'Region', None, None),
    ('Region with DTA', 'Region', None, None),
    ('ICB', 'ICB', None, None),
    ('ICB with DTA', 'ICB', None, None),
]

# === PROCESS EACH SHEET (all or nothing) ===
all_dfs = []


try:
    for sheet_name, geo_level, fixed_code, fixed_name in SHEETS:
        print(f"Preparing to process sheet: {sheet_name} for PATHWAY_TYPE: {PATHWAY_TYPE}")
        if PATHWAY_TYPE == 'DTA' and 'DTA' not in sheet_name:
            print(f"Skipping non-DTA sheet: {sheet_name}")
            continue
        if PATHWAY_TYPE != 'DTA' and 'DTA' in sheet_name:
            print(f"Skipping DTA-specific sheet: {sheet_name}")
            continue

        df = pd.read_excel(FILE_PATH, sheet_name=sheet_name, skiprows=13)
        # === REMOVE DUPLICATE NATIONAL ROWS from non-National sheets ===
        if sheet_name in ['Region', 'Region with DTA']:
            df = df[~((df['Region Name'].str.upper() == 'NHS ENGLAND') & (df['Region Code'] == '-'))]
        elif sheet_name in ['ICB', 'ICB with DTA']:
            df = df[~((df['ICB Name'].str.upper() == 'NHS ENGLAND') & (df['ICB Code'] == '-'))]


        percent_cols = [col for col in METRICS_LOOKUP[PATHWAY_TYPE] if '%' in col]
        for col in percent_cols:
            if col in df.columns:
                df[col] = (
                    df[col].astype(str)
                    .str.strip()
                    .replace('-', None)
                    .str.rstrip('%')
                    .astype(float)
                )

        # Assign codes and names
        if geo_level == 'National':
            df['org_code'] = fixed_code
            df['org_name'] = fixed_name
            df['region_code'] = None
        elif geo_level == 'Region':
            df['org_code'] = df['Region Code']
            df['org_name'] = df['Region Name']
            df['region_code'] = df['Region Code']
        elif geo_level == 'ICB':
            df['org_code'] = df['ICB Code']
            df['org_name'] = df['ICB Name']
            df['region_code'] = None  # To be joined later

        df['treatment_function_code'] = df['Treatment Function Code']
        df['treatment_function'] = df['Treatment Function']

        melted = df.melt(
            id_vars=['org_code', 'org_name', 'region_code', 'treatment_function_code', 'treatment_function'],
            value_vars=METRICS_LOOKUP[PATHWAY_TYPE],
            var_name='metric',
            value_name='value'
        )

        melted['value'] = pd.to_numeric(melted['value'], errors='coerce')
        melted['year'] = YEAR
        melted['month'] = MONTH
        melted['pathway_type'] = PATHWAY_TYPE
        melted['geo_level'] = geo_level

        all_dfs.append(melted)

    final_df = pd.concat(all_dfs, ignore_index=True)

    with engine.begin() as conn:
        final_df.to_sql('rtt_data', conn, if_exists='append', index=False)
    print(f"Loaded {len(final_df)} rows from {FILE_PATH}")

except Exception as e:
    print(f"Operation aborted. Error in sheet '{sheet_name}': {e}")


📄 Preparing to process sheet: National for PATHWAY_TYPE: DTA
⏭️ Skipping non-DTA sheet: National
📄 Preparing to process sheet: National with DTA for PATHWAY_TYPE: DTA
📄 Preparing to process sheet: Region for PATHWAY_TYPE: DTA
⏭️ Skipping non-DTA sheet: Region
📄 Preparing to process sheet: Region with DTA for PATHWAY_TYPE: DTA
📄 Preparing to process sheet: ICB for PATHWAY_TYPE: DTA
⏭️ Skipping non-DTA sheet: ICB
📄 Preparing to process sheet: ICB with DTA for PATHWAY_TYPE: DTA
✅ Loaded 2400 rows from ../data/commissioner/Incomplete_apr2023_march_2024/Incomplete-Commissioner-12-Mar24.xlsx
